In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.hierarchical_portfolio import HRPOpt
warnings.filterwarnings('ignore')

from AA import DataDownloader, AssetAllocation

In [2]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA'] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065
ff_factors_expectations = {'Mkt-RF': 0.05, 'SMB': 0.02, 'HML': 0.03, 'RF': 0.02}

asset_prices, benchmark_prices, ff_factors = downloader.download_data(start_date=start_date, end_date=end_date,
                                                                      assets=assets, benchmark=benchmark)

# Default limits for every asset (min 1% - max 100%):  boounds = tuple((0.01, 1) for _ in range(self.num_assets))
Asset_allocation = AssetAllocation(asset_prices=asset_prices, benchmark_prices=benchmark_prices, rf=rf, ff_factors=ff_factors) 
Asset_allocation.calculate_ff_expected_returns(ff_factors_expectations)

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Set Black-Litterman Expectations (Optional) 

P = np.array([
    [1, 0, 0, -1, 0],  
    [0, 1, -1, 0, 0],
    [0, 0, 0, 0, 1],
])

Q = np.array([0.05,  
              0.03,
              -0.15])  

Omega = np.diag([0.1**2, 0.15**2, 0.08**2])  # Incertidumbre en las vistas
tau = 0.08  # Incertidumbre en los rendimientos de equilibrio

Asset_allocation.set_blacklitterman_expectations(P, Q, tau, Omega)

### RMT Filter

In [4]:
#Matriz original de covarianzas
Asset_allocation.asset_cov_matrix

,AAPL,GOOG,IBM,NVDA,TSLA
AAPL,0.000412,0.000274,0.000147,0.000435,0.000404
GOOG,0.000274,0.000402,0.000132,0.000408,0.000333
IBM,0.000147,0.000132,0.000272,0.000180,0.000131
NVDA,0.000435,0.000408,0.000180,0.001064,0.000666
TSLA,0.000404,0.000333,0.000131,0.000666,0.001661


In [5]:
#
eigenvalues, eigenvectors = np.linalg.eigh(Asset_allocation.asset_cov_matrix)
eigenvalues, eigenvectors

(array([0.00012864, 0.00017918, 0.00028566, 0.0007526 , 0.00246516]),
 array([[ 0.79203973, -0.24343067, -0.39148111, -0.26202989, -0.30248545],
        [-0.58099927, -0.57790106, -0.40177392, -0.30510413, -0.27195478],
        [-0.17498467,  0.75054629, -0.59337155, -0.19635615, -0.12415756],
        [-0.04800429,  0.20494668,  0.57715689, -0.57904137, -0.53599764],
        [-0.04678531,  0.03806582, -0.01097191,  0.68147425, -0.7292699 ]]))

In [6]:
T, N = Asset_allocation.asset_returns.shape
T, N

(1257, 5)

In [7]:
sigma_squared = np.mean(np.diag(Asset_allocation.asset_cov_matrix))
sigma_squared 

0.0007622494334697608

In [8]:
lambda_plus = sigma_squared * (1 + np.sqrt(N/T))**2
lambda_plus

0.0008614303849375874

In [9]:
filtered_eigenvalues = np.clip(eigenvalues, 0, lambda_plus)
filtered_eigenvalues

array([0.00012864, 0.00017918, 0.00028566, 0.0007526 , 0.00086143])

In [10]:
filtered_eigenvalues - eigenvalues

array([ 0.        ,  0.        ,  0.        ,  0.        , -0.00160373])

In [11]:
# Matriz filtrada=Eigenvectores×Matriz Diagonal de Eigenvalores Filtrados×Eigenvectores 

filtered_cov_matrix = eigenvectors @ np.diag(filtered_eigenvalues) @ eigenvectors.T
pd.DataFrame(filtered_cov_matrix)

,0,1,2,3,4
0,0.000266,0.000142,0.000087,0.000175,0.000050
1,0.000142,0.000283,0.000078,0.000175,0.000015
2,0.000087,0.000078,0.000248,0.000074,-0.000015
3,0.000175,0.000175,0.000074,0.000603,0.000040
4,0.000050,0.000015,-0.000015,0.000040,0.000808


### Asset Allocation

In [12]:
optimizations = Asset_allocation.Optimize_Portfolio(method = "SLSQP")
optimizations

,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.212822,0.010000,0.010000,0.507455,0.259723,1.269909
Max (Smart) Sharpe,0.010000,0.010000,0.010000,0.569683,0.400317,1.200539
Max Sharpe Famma French,0.292493,0.010000,0.010000,0.010000,0.677507,1.510110
Max Omega,0.367593,0.018849,0.313470,0.213550,0.086538,1.348117
Max (Smart) Omega,0.010000,0.010000,0.201524,0.125332,0.653144,1.205929
Min VaR (Empirical),0.010000,0.010000,0.010000,0.389210,0.580790,-0.049680
Min VaR (Parametric),0.156482,0.217159,0.606359,0.010000,0.010000,-0.024424
Semivariance,0.200000,0.200000,0.200000,0.200000,0.200000,0.000401
Safety-First,0.221457,0.010000,0.010000,0.501680,0.256863,0.079996
Max Sortino,0.010000,0.010000,0.010000,0.747532,0.222468,0.115912


Usando el mismo timeframe y activos calculamos las metricas con los pesos datos por la optimizacion y verificamos el valor del porceso manual y la class AssetAllocation

### Sharpe 

In [13]:
Weights_Sharpe, Value_Sharpe = optimizations.loc["Max Sharpe"][:-1], optimizations.loc["Max Sharpe"][-1]
weights = Weights_Sharpe

#retornos de activos
returns = asset_prices.pct_change().dropna()
returns.head()

,AAPL,GOOG,IBM,NVDA,TSLA
Date,,,,,
2019-01-03,-0.099607,-0.028484,-0.019964,-0.060417,-0.031472
2019-01-04,0.042689,0.053786,0.039058,0.064068,0.057697
2019-01-07,-0.002226,-0.002167,0.007075,0.052941,0.054361
2019-01-08,0.019063,0.007385,0.014219,-0.024896,0.001164
2019-01-09,0.016982,-0.001505,0.007177,0.019667,0.009483


In [14]:
#retornos del portafolio
portfolio_returns = returns.dot(weights)
portfolio_returns.head()

Date
2019-01-03   -0.060516
2019-01-04    0.057510
2019-01-07    0.040559
2019-01-08   -0.008058
2019-01-09    0.016114
dtype: float64

In [15]:
#retornos menos la tasa libre de riesgo diaria
excess_returns_daily = portfolio_returns - (rf / 252)
excess_returns_daily


Date
2019-01-03   -0.060774
2019-01-04    0.057253
2019-01-07    0.040302
2019-01-08   -0.008316
2019-01-09    0.015856
                ...   
2023-12-22   -0.004947
2023-12-26    0.008062
2023-12-27    0.006081
2023-12-28   -0.006905
2023-12-29   -0.006271
Length: 1257, dtype: float64

In [16]:
# Anualizamos el promedio  de la resta para obtener rendimiento promedio anual del portafolio
excess_returns_annualized = excess_returns_daily.mean() * 252
excess_returns_annualized

0.5444138197692394

In [17]:
# Calculamos volatilidad anual del portafolio
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
portfolio_volatility 

0.42870304592858754

In [18]:
# Calculamos Sharpe
sharpe_ratio = excess_returns_annualized / portfolio_volatility
sharpe_ratio

1.2699089146661364

In [19]:
# Comparamos
round(sharpe_ratio - Value_Sharpe, 10)

-0.0

**Diferencia menor a $ 1e^{-10}$**

### Omega

In [20]:
Weights_Omega, Value_Omega = optimizations.loc["Max Omega"][:-1], optimizations.loc["Max Omega"][-1]
weights = Weights_Omega

weights

AAPL    0.367593
GOOG    0.018849
IBM     0.313470
NVDA    0.213550
TSLA    0.086538
Name: Max Omega, dtype: float64

In [21]:
# Retornos de activos
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

,0
Date,
2019-01-03,-0.059035
2019-01-04,0.047624
2019-01-07,0.017369
2019-01-08,0.006388
2019-01-09,0.013484
...,...
2023-12-22,-0.000629
2023-12-26,0.004396
2023-12-27,0.002715


In [22]:
# Calcular los retornos diarios del benchmark
benchmark_returns = benchmark_prices.pct_change().dropna()  
benchmark_returns 

,^GSPC
Date,
2019-01-03,-0.024757
2019-01-04,0.034336
2019-01-07,0.007010
2019-01-08,0.009695
2019-01-09,0.004098
...,...
2023-12-22,0.001660
2023-12-26,0.004232
2023-12-27,0.001430


In [23]:
# Obtener diferencia del portafolio respecto al benchmark
excess_returns = portfolio_returns[0] -  benchmark_returns[benchmark_returns.columns[0]] 
excess_returns

Date
2019-01-03   -0.034279
2019-01-04    0.013289
2019-01-07    0.010358
2019-01-08   -0.003307
2019-01-09    0.009386
                ...   
2023-12-22   -0.002289
2023-12-26    0.000164
2023-12-27    0.001285
2023-12-28   -0.001297
2023-12-29   -0.001203
Length: 1257, dtype: float64

In [24]:
positive_excess = excess_returns[excess_returns > 0].sum()
negative_excess = -excess_returns[excess_returns < 0].sum()
positive_excess, negative_excess

(4.787322032797221, 3.551117228381621)

In [25]:
omega_ratio = positive_excess / negative_excess
omega_ratio

1.3481171487484194

In [26]:
# Comparamos
round(omega_ratio - Value_Omega, 10)

0.0

**Diferencia menor a $ 1e^{-10}$**

### Safety First Ratio

In [27]:
Weights_SFRatio, Value_SFRatio = optimizations.loc["Safety-First"][:-1], optimizations.loc["Safety-First"][-1]
weights = Weights_SFRatio

weights

AAPL    0.221457
GOOG    0.010000
IBM     0.010000
NVDA    0.501680
TSLA    0.256863
Name: Safety-First, dtype: float64

In [28]:
# Retorno del portafolio
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

,0
Date,
2019-01-03,-0.060937
2019-01-04,0.057344
2019-01-07,0.040079
2019-01-08,-0.007753
2019-01-09,0.016120
...,...
2023-12-22,-0.004696
2023-12-26,0.008197
2023-12-27,0.006273


In [29]:
# Retorno del portafolio
portfolio_return = portfolio_returns.mean().item()
portfolio_return

0.0024078294015829516

In [30]:
# Retorno Mínimo Aceptable (MAR)
MAF = rf / 252
MAF

0.00025793650793650796

In [31]:
# Volatilidad del portafolio
cov_matrix = returns.cov()
portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
portfolio_vol

0.026875162458315446

In [32]:
# Calculamos el SFratio
SFratio = (portfolio_return - MAF) / portfolio_vol

print(f"Safety First Ratio: {SFratio}")

Safety First Ratio: 0.07999553107747802


In [33]:
# Comparamos
round(SFratio - Value_SFRatio, 10)

0.0

**Diferencia menor a $ 1e^{-10}$**

### HRP

In [34]:
Weights_HRP = optimizations.loc["HRP"][:-1]
weights = Weights_HRP
weights

AAPL    0.153590
GOOG    0.157631
IBM     0.506144
NVDA    0.099614
TSLA    0.083021
Name: HRP, dtype: float64

In [35]:
# Obtener pesos usando PyPortfolioOpt
hrp_pypfopt = HRPOpt(returns)
weights_pypfopt = hrp_pypfopt.optimize()
weights_pypfopt = pd.Series(hrp_pypfopt.clean_weights())

In [36]:
# DataFrame para comparar
comparison_df = pd.DataFrame({
    'Weights_HRP_Implementation': weights,
    'Weights_HRP_PyPortfolioOpt': weights_pypfopt
})
comparison_df = comparison_df.fillna(0)

In [37]:
# Calcular las diferencias
comparison_df['Differences'] = round(comparison_df['Weights_HRP_Implementation'] - comparison_df['Weights_HRP_PyPortfolioOpt'], 5)
comparison_df

,Weights_HRP_Implementation,Weights_HRP_PyPortfolioOpt,Differences
AAPL,0.153590,0.15359,0.0
GOOG,0.157631,0.15763,0.0
IBM,0.506144,0.50614,0.0
NVDA,0.099614,0.09961,0.0
TSLA,0.083021,0.08302,0.0


**Diferencia menor a $ 1e^{-5}$**

### Semivariance

In [41]:
Weights_Semivariance, Value_Semivariance = optimizations.loc["Semivariance"][:-1], optimizations.loc["Semivariance"][-1]
weights = Weights_Semivariance

weights

AAPL    0.2
GOOG    0.2
IBM     0.2
NVDA    0.2
TSLA    0.2
Name: Semivariance, dtype: float64

In [42]:
# Retorno
returns = asset_prices.pct_change().dropna()
asset_cov_matrix = returns.cov()

In [45]:
# Calculamos Semivarianza
semivariance = np.dot(weights, np.dot(asset_cov_matrix, weights))
print(f"Semivariance: {semivariance}")

Semivariance: 0.0004013791660417159


In [46]:
# Comparamos
round(semivariance - Value_Semivariance, 10)

-0.0

**Diferencia menor a $ 1e^{-10}$**